# Spam and Ham Email Detection
<div style="text-align: right;"><h4>Based on Natural Language Processing using PySpark's MLLib</h4></div>

#### Import libraries and explore dataset

In [1]:
import findspark
findspark.init('E:\DATA\Apps\hadoop-env\spark-2.4.7-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [6]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [7]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import length

data = data.withColumn('length', length(data['text']))
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [9]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



#### Feature Transformations

In [10]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer, IDF, StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

#### Build the model

In [12]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()

#### Build a pipeline

In [13]:
from pyspark.ml import Pipeline

data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [14]:
cleaner = data_prep_pipe.fit(data)

In [15]:
clean_data = cleaner.transform(data)

#### Training

In [16]:
clean_data = clean_data.select(['label','features'])

In [17]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [18]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [19]:
spam_predictor = nb.fit(training)

In [20]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [21]:
test_results = spam_predictor.transform(testing)

In [22]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-607.25257066215...|[1.0,1.0970249199...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1066.8861212973...|[1.0,1.0083149239...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-668.95395916693...|[1.0,4.0633823472...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.40073077459...|[1.0,8.8564075861...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-687.05820106334...|[1.0,3.0776808144...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-666.74013995653...|[1.0,2.0495680784...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-871.26962921920...|[1.0,7.0027547219...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1310.4144290175...|[1.0,8.9426504182...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-340.29083379504...|[1.0,3.9342981757...|       0.0|
|  0.0|(13424,[0

#### Evaluation

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9102461033058291
